In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os
proj = os.path.dirname(os.getcwd())
if proj not in sys.path:
    sys.path.insert(0, proj)
sys.path

['/Volumes/Projects/Evidently/record-thing',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python311.zip',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/lib-dynload',
 '',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages']

In [3]:
# Device independent code
import torch

device = "cpu" 
if torch.cuda.is_available():
    device = "cuda"
# if torch.backends.mps.is_available:
#     device = torch.device("mps")
torch.device(device)
torch.set_default_device(device=device)

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [4]:
!docker compose up -d

/Users/henrikvendelbo/.zshenv:.:1: no such file or directory: /Users/henrikvendelbo/.cargo/env
[+] Running 1/0
 ✔ Container rqlite_server  Running                                        0.0s 


In [5]:
from dataset.db import init_db_sqlite
connection = init_db_sqlite(disconnect=False)
connection

# UPC/QR lookup

If the scanner recognises a barcode or QR code the product number should be looked up.
If multiple matches are found the category of product should be used to pick the most likely match.
The user is asked to confirm the match YES/NO.

In [9]:
from dataset.product import lookup_product
from dataset.ingestion import Ingestor
import pandas as pd

upc = '0885909950805'
with Ingestor(connection.cursor(), device=device) as ingestor:

    result = lookup_product(upc, ingestor=ingestor, save=True)
    if result:
        print(result)
        display(pd.DataFrame([result], columns=["id", "name", "upc","asin","elid","brand","model","color","tags","category","title","description", "account"]))

        # match with product["items"][0]["images"] product similarity with pictures of the recording.
        # print(number_matches, upc, item)

Product found: (b'2puGqVPJ8acvaiBzY5kxcwZATLX', b'2puGrC5nHEndsfLjQ8GqXGrikaT', '885909950805', 'B00NQGOZV0', '195389220744', 'Apple', 'MG5A2LL/A', 'Space Gray', ',,', 'Electronics > Communications > Telephony > Mobile Phones > Unlocked Mobile Phones', 'Apple iPhone 6, Space Gray, 64 GB (T-Mobile)', 'Display   Retina HD display 1334-by-750-pixel resolution at 326 ppi 1400:1 contrast ratio (typical) 500 cd/m2 max brightness (typical) Full sRGB standard Dual-domain pixels for wider viewing angles Fingerprint-resistant oleophobic coating on front Support for display of multiple languages and characters simultaneously Display Zoom iSight Camera Autofocus with Focus Pixels f/2.2 aperture True Tone flashHybrid IR filter Backside illumination sensor Sapphire crystal lens cover Auto image stabilization Auto HD...', 'Apple iPhone 6, Space Gray, 64 GB (T-Mobile)')
Product found: id=b'2puGrC5nHEndsfLjQ8GqXGrikaT' account=b'2puGqVPJ8acvaiBzY5kxcwZATLX' upc='885909950805' asin='B00NQGOZV0' elid='19

,id,name,upc,asin,elid,brand,model,color,tags,category,title,description,account
0,"(id, b'2puGrC5nHEndsfLjQ8GqXGrikaT')","(account, b'2puGqVPJ8acvaiBzY5kxcwZATLX')","(upc, 885909950805)","(asin, B00NQGOZV0)","(elid, 195389220744)","(brand, Apple)","(model, MG5A2LL/A)","(color, Space Gray)","(tags, ,,)","(category, Electronics > Communications > Tele...","(title, Apple iPhone 6, Space Gray, 64 GB (T-M...","(description, Display Retina HD display 1334...","(name, Apple iPhone 6, Space Gray, 64 GB (T-Mo..."


In [10]:
import pandas as pd

db = connection

# with connection.cursor() as cursor:
count = db.execute(
"""
  SELECT COUNT(*) FROM products
"""
        ).fetchone()

display(f"clip_assets count: {count[0]}")

rows = db.execute(
    """
      SELECT id, name, upc, asin, elid, brand, model, color, tags, category, title, description FROM products
      LIMIT 30
    """
    ).fetchall()


display(pd.DataFrame(rows, columns=["id", "name", "upc","asin","elid","brand","model","color","tags","category","title","description"]))


/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sqlmodel/main.py:638: SAWarning: This declarative base already contains a class with the same class name and module name as dataset.db.products.Products, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)
[autoreload of dataset.db.products failed: Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib

'clip_assets count: 1'

,id,name,upc,asin,elid,brand,model,color,tags,category,title,description
0,b'2puGrC5nHEndsfLjQ8GqXGrikaT',"Apple iPhone 6, Space Gray, 64 GB (T-Mobile)",885909950805,B00NQGOZV0,195389220744,Apple,MG5A2LL/A,Space Gray,",,",Electronics > Communications > Telephony > Mob...,"Apple iPhone 6, Space Gray, 64 GB (T-Mobile)",Display Retina HD display 1334-by-750-pixel ...


In [8]:
from typing import Sequence
import torch
import requests

def generate_embedding(image: bytes) -> torch.Tensor:
    # dino.
    pass

def get_images(image_urls: Sequence[str]) -> Sequence[bytes]:
    images: Sequence[bytes] = [requests.get(url).content for url in image_urls]
    embeddings: Sequence[torch.Tensor] = [generate_embedding(image) for image in images]